In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini", trigger=("tokens", 100), keep=("messages", 1)
        )
    ],
)

In [ ]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {
        "messages": [
            HumanMessage(content="What is the capital of the moon?"),
            AIMessage(content="The capital of the moon is Lunapolis."),
            HumanMessage(content="What is the weather in Lunapolis?"),
            AIMessage(
                content="Skies are clear, with a high of 120C and a low of -100C."
            ),
            HumanMessage(content="How many cheese miners live in Lunapolis?"),
            AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
            HumanMessage(content="Do you think the cheese miners' union will strike?"),
            AIMessage(content="Yes, because they are unhappy with the new president."),
            HumanMessage(
                content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"
            ),
        ]
    },
    {"configurable": {"thread_id": "1"}},
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\n## SESSION INTENT\nThe user is inquiring about various details related to Lunapolis, the capital of the moon, including its weather, population of cheese miners, and the potential for a strike by the cheese miners' union.\n\n## SUMMARY\n- The capital of the moon is Lunapolis.\n- The weather in Lunapolis is clear with a high of 120°C and a low of -100°C.\n- There are 100,000 cheese miners residing in Lunapolis.\n- The cheese miners' union is likely to strike due to dissatisfaction with the new president.\n\n## ARTIFACTS\nNone\n\n## NEXT STEPS\nNone", additional_kwargs={'lc_source': 'summarization'}, response_metadata={}, id='2660a1ad-d84d-4705-9993-9a37b8a49c6b'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='a590c460-91cd-4679-a997-02fb1ed0b908'),
              AIMessage(cont

In [4]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

## SESSION INTENT
The user is inquiring about various details related to Lunapolis, the capital of the moon, including its weather, population of cheese miners, and the potential for a strike by the cheese miners' union.

## SUMMARY
- The capital of the moon is Lunapolis.
- The weather in Lunapolis is clear with a high of 120°C and a low of -100°C.
- There are 100,000 cheese miners residing in Lunapolis.
- The cheese miners' union is likely to strike due to dissatisfaction with the new president.

## ARTIFACTS
None

## NEXT STEPS
None


## Trim/delete messages

In [ ]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage


@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]

    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [6]:
agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [ ]:
response = agent.invoke(
    {
        "messages": [
            HumanMessage(content="My device won't turn on. What should I do?"),
            ToolMessage(
                content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"
            ),
            AIMessage(content="Is the device plugged in and turned on?"),
            HumanMessage(content="Yes, it's plugged in and turned on."),
            ToolMessage(
                content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"
            ),
            AIMessage(content="Is the device showing any lights or indicators?"),
            HumanMessage(content="What's the temperature of the device?"),
        ]
    },
    {"configurable": {"thread_id": "2"}},
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='163f5e69-fb1b-418d-82b5-4a651dd347c7'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='fcf085e7-af99-46c0-9e6b-24ec3bf523da', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='25d2d3bc-e768-423b-9666-d50150887048'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='50fe1e5d-b370-4021-82c0-a6440da3e146', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='d6e1a586-e484-4d17-8914-af0878088bac'),
              AIMessage(content='I don’t have a way to read the device’s internal temperature f

In [8]:
print(response["messages"][-1].content)

I don’t have a way to read the device’s internal temperature from here. You can check it manually:

- Feel the exterior: if it’s very hot to the touch (hot enough that you can’t keep your finger on it for more than a second), it’s overheating.
- If you have an infrared/thermal thermometer, scan near the vents to get a rough surface temp. A safe surface is generally warm but not painfully hot; anything well above 50–60°C (or uncomfortably hot to touch) is a sign to cool it down.

What to do right now if it’s overheating:
- Power it off and unplug it.
- Let it cool on a hard surface with good airflow for 15–30 minutes.
- While cooling, check vents and fans for dust and ensure nothing blocks airflow.
- After it has cooled, try turning it on again. If it overheats again quickly, there may be a cooling/fan issue or a hardware fault.

If you can tell me what type of device (laptop, desktop PC, tablet, smartphone) and any indicators (lights, beeps), I can give more targeted steps.
